In [ ]:
from qiskit import QuantumProgram, QuantumCircuit, ClassicalRegister, QuantumRegister, QISKitError
from qiskit import available_backends, register, execute, get_backend
from qiskit.tools.visualization import circuit_drawer, plot_histogram, plot_state
from qiskit.tools.qi.qi import state_fidelity
import random, pickle, time, datetime, xlsxwriter
import scipy.io as spio
import importlib
import circuit_builder
importlib.reload(circuit_builder)
import encoding_schemes
importlib.reload(encoding_schemes)

#setup
import sys, getpass
try:
    sys.path.append("../../") # go to parent dir
    import Qconfig
    qx_config = {
        "APItoken": Qconfig.APItoken,
        "url": Qconfig.config['url']}
    print('Qconfig loaded from %s.' % Qconfig.__file__)
    print()
except:
    APItoken = getpass.getpass('Please input your token and hit enter: ')
    qx_config = {
        "APItoken": APItoken,
        "url":"https://quantumexperience.ng.bluemix.net/api"}
    print('Qconfig.py not found in qiskit-tutorial directory; Qconfig loaded using user input.')
    
#matlab stuff
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from numpy import matrix
from math import pi

#set 
QX_URL = "https://quantumexperience.ng.bluemix.net/api"
QX_TOKEN = "06704f99fc6dd68ab5ae2663f0861723406de8b4553cacdabdf500292c937717442daa90c2ffa19109cfb9114adb7c87711a08860d34d1da9332ac664e4e24a7"

#register token and url
register(QX_TOKEN, QX_URL)

# query for available backends
print("Available backends: ", available_backends())

backend = "local_qasm_simulator"


def getReg(qc,k):
    desired = list(qc.get_qregs().items())
    return desired[k][1]

scheme = encoding_schemes.FiveOneThree()

n = scheme.n
validOutputs = scheme.validOutputs
perm = scheme.perm
scheme.setPerm()

qr = QuantumRegister(n)
cr = ClassicalRegister(n)
qc = QuantumCircuit(qr,cr)



scheme.prepareZeros(qc)

scheme.setInput(qc,0)

scheme.measure(qc,0,False)





Qconfig loaded from C:\Users\ddrsq\OneDrive\Quantum\qecc_benchmark\Qconfig.py.

